In [39]:
import pandas as pd
import sqlite3 as sql
import requests
import re
import json
from bs4 import BeautifulSoup as bs

In [40]:
base_url="https://insider.in"
url="https://insider.in/all-events-in-pune"
r=requests.get(url)
c=r.content
conn=sql.connect("Events.db")
cur=conn.cursor()

In [41]:
soup=bs(c,"html.parser")
cur.execute("CREATE TABLE IF NOT EXISTS int_url2 (id INTEGER PRIMARY KEY, EventURL TEXT, Name TEXT, EventDescription TEXT, PerformerName TEXT, EventStart TEXT, EventEnd TEXT, Location TEXT, Address TEXT, Price TEXT)")
cur.execute("SELECT * FROM int_url2")
try:
    cur.execute("CREATE UNIQUE INDEX int_url2_Name on int_url2 ( Name )")
except:
    pass
print(*cur.fetchall(), sep='\n')

In [42]:
"""Crawler to scrape all the urls from page and visit them"""
int_url = []
non_int_url = []
for link in soup.findAll('a'):
    str1=str(link.get('href'))
    if str1.endswith('/event') and str1.startswith('https:') and len(int_url)<10 and str1 not in int_url:
        int_url.append(link.get('href'))
    elif str1.endswith('/event') and len(int_url)<10:
        int_url.append(base_url+link.get('href'))
    elif len(int_url)==10:
        break
    else:
        non_int_url.append(base_url+link.get('href'))
df=pd.DataFrame()
df["Non Interesting URLs"]=pd.Series(non_int_url)
df["Interesting URLs"]=pd.Series(int_url)
df

,Non Interesting URLs,Interesting URLs
0,https://insider.in/pune,https://insider.in/india-bike-week-2019-dec6-7...
1,https://insider.in/movies,https://insider.in/oneplus-music-festival-nov1...
2,https://insider.in/gameshows,https://insider.in/zomaland-pune-2019/event
3,https://insider.in/monuments,https://insider.in/bacardi-nh7-weekender-pune-...
4,https://insider.in/all-events-in-pune,https://insider.in/india-bike-week-2019-dec6-7...
5,https://insider.in/all-amusement-parks-in-pune,https://insider.in/supermoon-ft-prateek-kuhad-...
6,https://insider.in/all-travel-in-pune,https://insider.in/prashasti-singh-and-samay-r...
7,https://insider.in/all-workshops-in-pune,https://insider.in/making-money-for-my-kids-by...
8,https://insider.in/all-theatre-in-pune,https://insider.in/oktoberfest-drinkathon-carn...
9,https://insider.in/artists,https://insider.in/mentalism-special-by-suhani...


In [43]:
""" Name, Performer,Description,startdate,enddate, Location, Price(low-high) """
l= []
for event in int_url:
    d= {}
    r=requests.get(event)
    c=r.content
    soup=bs(c,'html.parser')
    data = soup.find_all('script',{'type':'application/ld+json'})[0]
    oJson = json.loads(data.text)
    d['Event URL']=event
    d['Event Name']=oJson['name']
    d['Event Description']=bs(oJson['description']).text
    d['Performer Name']=oJson['performer']['name']
    d['Event Start DateTime']=oJson['startDate']
    d['Event End DateTime']=oJson['endDate']
    d['Location']=oJson['location']['name']
    d['Address']=oJson['location']['address']
    d['Price']='INR '+str(oJson['offers']['lowPrice'])+'-'+str(oJson['offers']['highPrice'])
    l.append(d)
    try:
        cur.execute("INSERT INTO int_url2 VALUES ( NULL, ?, ?, ?, ?, ?, ?, ?, ?, ?)",(d["Event URL"],d["Event Name"],d["Event Description"],d["Performer Name"],d["Event Start DateTime"],d["Event End DateTime"],d["Location"],d["Address"],d["Price"]))
    except:
        pass
df=pd.DataFrame(l)
df

,Event URL,Event Name,Event Description,Performer Name,Event Start DateTime,Event End DateTime,Location,Address,Price
0,https://insider.in/india-bike-week-2019-dec6-7...,India Bike Week 2019,It's more than a hobby.It's more than a passio...,Seventy Emg,2019-12-06T08:30:00.000Z,2019-12-07T16:30:00.000Z,"Little Vagator Hills, Bardez, Goa","Ozran Beach Road, Bardez, Vagator, Goa 403509",INR 600-2500
1,https://insider.in/oneplus-music-festival-nov1...,"OnePlus Music Festival, 2019",A mix of prime international acts in a concert...,OnePlus,2019-11-16T08:30:00.000Z,2019-11-16T16:30:00.000Z,D.Y. Patil Stadium,"Dy Patil Vidyanagar Campus 4 B Sector 7, Nerul...",INR 10-200000
2,https://insider.in/zomaland-pune-2019/event,Zomaland - Pune,"Come one, come all\nTo the grandest carnival o...",Zomato,2019-11-15T06:30:00.000Z,2019-11-15T16:30:00.000Z,"Royal Palms, Pune","84/36, N Main Rd, Koregaon Park Annexe, Mundhw...",INR 99-599
3,https://insider.in/bacardi-nh7-weekender-pune-...,"Bacardi NH7 Weekender 2019, Pune","​The happiest music festival is back, and we’r...",OML,2019-11-29T09:30:00.000Z,2019-12-01T16:30:00.000Z,"Mahalakshmi Lawns, Pune, Airport Area","Nagar Road, Near Kharadi Bypass, Opp Lohagaon-...",INR 0-5500
4,https://insider.in/india-bike-week-2019-dec6-7...,India Bike Week 2019,It's more than a hobby.It's more than a passio...,Seventy Emg,2019-12-06T08:30:00.000Z,2019-12-07T16:30:00.000Z,"Little Vagator Hills, Bardez, Goa","Ozran Beach Road, Bardez, Vagator, Goa 403509",INR 600-2500
5,https://insider.in/supermoon-ft-prateek-kuhad-...,Supermoon ft. Prateek Kuhad Winter Tour 2019 -...,Abhi toh Pune show baaki hai...aur Raat Raazi ...,Zee Entertainment Enterprises Ltd.,2019-12-12T14:30:00.000Z,2019-12-12T17:30:00.000Z,"Royal Palms, Pune","84/36, N Main Rd, Koregaon Park Annexe, Mundhw...",INR 750-6500
6,https://insider.in/prashasti-singh-and-samay-r...,Prashasti Singh and Samay Raina - Live in Pune!,"See one of country's top comic, supported by a...",Spade Entertainment,2019-11-14T14:30:00.000Z,2019-11-14T16:00:00.000Z,Spade Comedy Club,"Kogobo Resto Bar & Banquet Baner, 4th Floor, T...",INR 499-499
7,https://insider.in/making-money-for-my-kids-by...,Making Money For My Kids by Sorabh Pant,Sorabh Pant tries new jokes to try and make mo...,OML,2019-11-15T14:30:00.000Z,2019-11-15T15:50:00.000Z,Spade Comedy Club,"Kogobo Resto Bar & Banquet Baner, 4th Floor, T...",INR 299-399
8,https://insider.in/oktoberfest-drinkathon-carn...,Oktoberfest Drinkathon Carnival 1.0,Gypsy Soul in association with Independence Br...,Gypsy Soul,2019-11-16T08:30:00.000Z,2019-11-17T04:30:00.000Z,"Gypsy Soul, Pune","Panshet, Maharashtra 412107",INR 849-2199
9,https://insider.in/mentalism-special-by-suhani...,Mentalism Special by Suhani Shah: Pune,Suhani Shah is one of the biggest name in Indi...,OML,2019-11-16T13:00:00.000Z,2019-11-16T14:30:00.000Z,Classic Rock Kalyaninagar,"Mulik Capital, Kalyani Nagar, Pune, Maharashtr...",INR 499-499


In [44]:
cur.execute("SELECT * FROM int_url2")
rows=cur.fetchall()
print(*rows, sep = "\n") 


(1, 'https://insider.in/india-bike-week-2019-dec6-7-19/event', 'India Bike Week 2019', "It's more than a hobby.It's more than a passion.In India, biking has achieved a new state. It's wild.India Bike Week embodies this love in one insane weekend of revelry, bonding and respect.And this year we're calling everyone from the casual to the hardcore, from the non-stop ravers to the adrenaline cravers and all those in between, to their true home land.A land where character is currency.And where every journey finds one hell of a destination.Welcome to the land of the wild.India Bike Week 2019\n\nKTM Orange Pass – Exclusive offer for KTM Riders. Visit ktmindiabikeweek.com\nHighlights:\nFlat Track Timed TrialHill Climb TrialsEnduro TrackFMX Show and StuntsBike and Brand LaunchesIBW Bike Build OffBikers Mart – India’s biggest biking expoMod-Bike, Barn DisplayBig Trip SpeakersHonour RollIBW Motovlogger MeetIBW Club VillageMoto MobMoto FashionMoto ArtIBW Film FestivalContests - Beard & Tattoo Cont

In [45]:
conn.commit()
conn.close()

df.to_csv('insider.csv')